<a href="https://colab.research.google.com/github/djdunc/casa0018/blob/main/Week7/CASA0018_7_lab_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Workshop 7 - Time Series Forecasting with Recurrent Neural Networks**




First set up the necessary Python imports.

In [ ]:
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator # Generates batches for sequence data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import files
import tensorflow as tf




# Set Seeds

In [ ]:
# Set seed for reproducibility
seed = 1234
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.random.set_random_seed(1234)

# Load sensor data from csv file

In [ ]:

from google.colab import files
uploaded = files.upload()


# Load into a dataframe

In [ ]:
df = pd.read_csv('light-2000.csv')
print(len(df))
print(df)
df.plot()


Split the data into train and test (10%) subsets.So rather than splitting the data into train and test dataset using traditional train_test_split function from sklearn, here we’ll split the dataset using simple python libraries to better understand the process going under the hood.
First, we’ll check the length of the data frame and use 10 percent of the training data to test our model. Now if we multiply the length of the data frame with test_percent and round the value (as we are using for indexing purpose) we’ll get the index position i.e., test_index. Last, we’ll split the train and test data using the test_index.

In [ ]:
print(len(df)) # 2001
test_percent = 0.1 # 10 percent of data
len(df)*test_percent # 200.1
test_point = np.round(len(df)*test_percent) # 200
test_index = int(len(df) - test_point) #1801
train = df.iloc[:test_index]
test = df.iloc[test_index:]

print(len(train))
print(len(test))

#plt.xlim(0, 50)
#plt.ylim(-5, 5)
#train.plot()
#test.plot()
plt.plot(train)
plt.plot(test)


We need to normalise the data in the range 0-1. We use a scaler to determine the max and min of the training set and then use the scale to scale both the training a test data set

In [ ]:
scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)


One problem we’ll face when using time series data is, we must transform the data into sequences of samples with input data and target data before feeding it into the model. We should select the length of the sequence data (window length) in such a way so that the model has an adequate amount of input data to generalize and predict.

The model takes the previous 100 data points (one cycle) as input data and uses it to predict the next point, which is then compared to the actual target value for backpropagation and gradient descent. This process is time-consuming and difficult if we perform this manually, hence we’ll make use of the Keras Timeseries Generator which transforms the data automatically and ready to train models without heavy lifting.


In [ ]:
length =  100 #sequence length - the length of the training window
batch_size=10
generator = TimeseriesGenerator(data=scaled_train, targets=scaled_train, length=length, batch_size=batch_size)
validation_generator = TimeseriesGenerator(data=scaled_test, targets=scaled_test, length=length, batch_size=batch_size)
print(len(scaled_train)) 
print(len(scaled_test))
print(generator.length) 
print(validation_generator.length)
print(len(generator.data)) 
print(len(validation_generator.data))

Create a model and train it. The variable (n_features) defined stands for the number of features in the training data i.e., as we are dealing with univariate data we’ll only have one feature whereas if we are using data containing multiple features then we must specify the count of features in our data.

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

n_features = 1
rnn_model = Sequential()
output_space = length # Same as number of time steps in the training window

rnn_model.add(SimpleRNN(output_space, return_sequences=False, input_shape = (length , n_features)))

#rnn_model.add(SimpleRNN(output_space, return_sequences=True, input_shape = (length , n_features)))
#rnn_model.add(SimpleRNN(output_space))

rnn_model.add(Dense(1))

rnn_model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001), loss='mse')
rnn_model.fit(generator, epochs=200, validation_data=validation_generator, callbacks=[callback])


# Testing

Let’s test our model using first_eval_batch. The first_eval_batch contains the last 50 points of the scaled training data and uses these to make a prediction. The results of the predicted value and the first observation in the scaled_data is outputted below. 

In [ ]:
first_eval_batch = scaled_train[-length:] # Take the last 100 points and predict the new value in the scaled_test
first_eval_batch = first_eval_batch.reshape((1, length, n_features)) # shape the data to match the input_shape of model
print(rnn_model.predict(first_eval_batch)) # array([[0.92780817]], dtype=float32)
print(scaled_test[0]) # array([0.94955134])

We can also look further into the future.

First, we’ll define an empty list (test_predictions) so we can append the predicted values. The second step is to define first_eval_batch i.e. the first evaluation batch that needs to be sent into the model and reshape the batch so it matches the input shape of our model. Our current_batch contains the last 100 values from the training data.

Finally, we’ll define a loop that continues until it reaches the end of the test data. The predicted value gets appended to the end of current_batch and the first observation in the current_batch gets removed. 


In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]

current_batch = first_eval_batch.reshape(1, length, n_features)


for i in range(len(test)):
  current_pred = rnn_model.predict(current_batch)[0]

  test_predictions.append(current_pred)

  current_batch = np.append(current_batch[:, 1:, :],[[current_pred]],axis = 1)

true_predictions = scaler.inverse_transform(test_predictions)
test['RNN Predictions'] = true_predictions
test.plot(figsize=(12,8))
#test.plot()





#**Long Short-Term Memory (LSTM) Neural Networks**

Now lets try a LSTM model. The code is very similar to that for a vanilla RNN except we use an LSTM layer.

In [ ]:
length = 100
batch_size = 10
n_features = 1

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

generator = TimeseriesGenerator(scaled_train, scaled_train, length=length, batch_size=batch_size)
validation_generator = TimeseriesGenerator(scaled_test, scaled_test, length=length, batch_size=batch_size)

lstm_model = Sequential()

output_space = length # Same as number of time steps in the training window

lstm_model.add(LSTM(output_space, input_shape=(length, n_features)))

#lstm_model.add(LSTM(output_space, return_sequences=True, input_shape=(length,n_features)))
#lstm_model.add(LSTM(output_space))

lstm_model.add(Dense(1))

lstm_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse')
lstm_model.fit(generator, epochs=200, validation_data=validation_generator, callbacks=[callback])


**Predict and Visualise**

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]

current_batch = first_eval_batch.reshape(1, length,n_features)


for i in range(len(test)):
  current_pred = lstm_model.predict(current_batch)[0]

  test_predictions.append(current_pred)

  current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis = 1)

true_predictions = scaler.inverse_transform(test_predictions)
test['LSTM Predictions'] = true_predictions
test.plot(figsize=(12,8))

#**Forecasting**
Let's do some  real forecasting, beyond the end of all our data (training+validation)


 
**RNN Forecasting**

In [ ]:
full_scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
scaled_full_data = full_scaler.fit_transform(df)

forecast = []

first_eval_batch = scaled_full_data[-length:]

current_batch = first_eval_batch.reshape(1, length, n_features)


for i in range(100):
  current_pred = rnn_model.predict(current_batch)[0]

  forecast.append(current_pred)

  current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis = 1)


**Plot the RNN Forecasted values**

In [ ]:
forecast = full_scaler.inverse_transform(forecast)

forecast_index = np.arange(2001, 2101, step=1)

plt.figure(figsize=(12,8))
plt.plot(df.index, df['light'])
plt.plot(forecast_index, forecast)
plt.show()



**LSTM Forecasting**

In [ ]:
forecast = []

first_eval_batch = scaled_full_data[-length:]

current_batch = first_eval_batch.reshape(1, length, n_features)


for i in range(100):
  current_pred = lstm_model.predict(current_batch)[0]

  forecast.append(current_pred)

  current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis = 1)


**Plot the LSTM Forecasted Values**

In [ ]:
forecast = full_scaler.inverse_transform(forecast)

forecast_index = np.arange(2001, 2101, step=1)

plt.figure(figsize=(12,8))
plt.plot(df.index, df['light'])
plt.plot(forecast_index, forecast)
plt.show()